# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [ ]:
# # 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)
# drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [ ]:
# 구글 드라이브에 업로드된 대회 데이터를 압축 해제하고 로컬에 저장합니다.
# !tar -xvf drive/MyDrive/datasets_fin.tar > /dev/null

In [ ]:
# 필요한 라이브러리를 설치합니다.
# !pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.0 MB/s eta 0:00:00


## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [3]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [6]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train() # model을 train mode로 변경
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device) # image를 gpu
        targets = targets.to(device)  # targets를 gpu로

        model.zero_grad(set_to_none=True) # 무슨 model인지 모르겠지만 model gradient 초기화
        # set_to_none = False가 default
        # set_to_none = True인 경우 gradient를 0으로 초기화하는 것이 아닌 None으로 초기화 메모리 사용량이 매우 감소하게됨
        # 일부 연산에서 None, 0인 경우 결과가 달라지지만 일반적인 SGD, Adam등에서는 문제가 없음
        preds = model(image) # model forward 수행

        loss = loss_fn(preds, targets) 
        # loss는 파이토치의 tensor type임 이 값은 autograd 그래프에 연결된 상태여서 그대로 더하게 되면 파이썬 float이 아니라
        # tensor가 계속 누적되게 됨 결과적으로 train_loss가 tensor로 변환되고 연산 그래프가 쌓여서 메모리 낭비가 되어
        # backward 시 불필요한 계산이 붙을 수 있음
        # loss.item()으로 하면 그냥 숫자만 뽑아오기때문에 문제 x 

        # torch에서 제공하는 내장 loss function에 경우 logit과 클래스 인덱스 스칼라값을 기입하면 알아서 계산해서 loss값을 제공함
        # loss function을 통해 loss score 계산 
        # 기존에 존재하는 모형을 불러왓으면 pred가 잘 수행됨
        loss.backward() # loss.backward를 통해 각 파라미터에 gradient 계산
        optimizer.step() # backward에서 계산된 gradient를 통해 parameter update 

        train_loss += loss.item() 
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy()) # 예측값을 계산
        # 굳이 softmax할필요없이 가장 큰값이 pred 값이 됨 
        targets_list.extend(targets.detach().cpu().numpy()) #  타겟값을 저장  gpu에 올라가있는 scarlar값이기 때문에 

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader) # epoch가 한번 돌면 train_loss에 loader에 길이만큼 나누어주게 됨 마지막 loader에서 기존의 batchsize와는 다르게
    # 짧은 값이 연산되므로 조금 값이 정확하지 않을 수 있음 
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro') # scikit-learn에 있는 f1_score 활용 

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [14]:
# 전반적인 학습에 필요한 초모수 정의 

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cuda' if torch.cuda.is_available() else 'cpu' 이런식으로 코드를 작성해도되지만 torch.device 객체로
# 선언되는게 공식문서에서도 추천하는 방식이고 훨씬 안전함 추가로 cuda:0 device 객체는 타입과 인덱스를 명확히 구분해져있기 때문에 더욱 권장된다.

# data config
data_path = '/root/dev/data/' # data가 저장되어있는  data path

# model config
model_name = 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config # training에 필요한 여러가지 초모수들
img_size = 256
LR = 1e-3
EPOCHS = 30
BATCH_SIZE = 32
num_workers = 0

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [8]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([ # 증강을 위한 
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # resnet 모형에 잘 알려진 평균과 분산들
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [11]:
# Dataset 정의
trn_dataset = ImageDataset(
    "/root/dev/data/train.csv", # trainset에 대한 meta data 
    "/root/dev/data/train/", # 실제 이미지가 어디에 있는지에 대한 path 
    transform=trn_transform # 위에서 지정한 transform
)
tst_dataset = ImageDataset(
    "/root/dev/data/sample_submission.csv", # testset에 대한 meta data target은 0으로 dummy임
    "/root/dev/data/test/", # test image가 어디에 있는지 경로
    transform=tst_transform # 위에서 지정한 transform 
)

# 중요하게 여겨야하는 것 중 하나는 모형의 성능을 위한 전략 중 주요한 것 중 하나는 augmentation이다.
# augmentation을 통해 데이터를 다양하게 증강시키는 것은 어디까지나 train set에 한해서 행해지는 방법이다.
# test set에는 이러한 것들이 적용되면 안됨 단순히 resize, 정규화랑, tensor 변환과는 다른 문제이다.

print(len(trn_dataset), len(tst_dataset))

1570 3140


In [12]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [13]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [15]:
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 1.8228: 100%|██████████| 50/50 [00:07<00:00,  7.00it/s]


train_loss: 1.6981
train_acc: 0.5643
train_f1: 0.5164
epoch: 0.0000



Loss: 1.1849: 100%|██████████| 50/50 [00:06<00:00,  8.22it/s]


train_loss: 0.4033
train_acc: 0.8739
train_f1: 0.8487
epoch: 1.0000



Loss: 0.8964: 100%|██████████| 50/50 [00:06<00:00,  8.24it/s]


...

train_loss: 0.0251
train_acc: 0.9962
train_f1: 0.9965
epoch: 27.0000



Loss: 0.7127: 100%|██████████| 50/50 [00:05<00:00,  8.36it/s]


train_loss: 0.0558
train_acc: 0.9866
train_f1: 0.9847
epoch: 28.0000



Loss: 0.9777: 100%|██████████| 50/50 [00:06<00:00,  8.31it/s]

train_loss: 0.0420
train_acc: 0.9930
train_f1: 0.9929
epoch: 29.0000



# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [16]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:08<00:00, 11.38it/s]


In [17]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [18]:
sample_submission_df = pd.read_csv("/root/dev/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [19]:
pred_df.to_csv("pred_epoch30_size256.csv", index=False)

In [20]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,6
2,00396fbc1f6cc21d.jpg,9
3,00471f8038d9c4b6.jpg,0
4,00901f504008d884.jpg,2
